In [1]:
# product_id 等置0
import pandas as pd
from constants import DATA_DIR, HEADER_TEST_SAMPLE

In [2]:
dataset = pd.read_csv(DATA_DIR + 'dataset.tsv', sep='\t')

In [3]:
to_rep_cols = [col for col in HEADER_TEST_SAMPLE if col in dataset.columns]
dataset[to_rep_cols] = dataset[to_rep_cols].replace(-1, 0)
dataset.head()

,ad_id,next_day,label,account_id,altered,bid,create_ts,crow,exposure_time,industry_id,op_date,op_day,product_class,product_id,sizes
0,10003,20190222,2,6539,0,31,1547593423,area:7572,"281474976694272,281474976694272,28147497669427...",186,2019-02-21,20190115,13,9446,30
1,100046,20190309,2,22818,0,150,1546206412,area:3269,"281474976694272,281474976694272,28147497669427...",17,2019-03-08,20181230,13,20433,34
2,100046,20190310,1,22818,0,150,1546206412,area:3269,"281474976694272,281474976694272,28147497669427...",17,2019-03-09,20181230,13,20433,34
3,100046,20190314,1,22818,0,150,1546206412,area:3269,"281474976694272,281474976694272,28147497669427...",17,2019-03-13,20181230,13,20433,34
4,100091,20190219,5,1385,0,31,1546292099,area:7572,"281474976694272,281474976694272,28147497669427...",84,2019-02-18,20181231,13,24175,30


In [4]:
dataset.dtypes

ad_id             int64
next_day          int64
label             int64
account_id        int64
altered           int64
bid               int64
create_ts         int64
crow             object
exposure_time    object
industry_id       int64
op_date          object
op_day            int64
product_class     int64
product_id        int64
sizes             int64
dtype: object

In [5]:
# 加入上线时间特征
dataset['online_days'] = (pd.to_datetime(dataset.op_date)  - pd.to_datetime(dataset.create_ts, unit='s')).dt.days
dataset.head()

,ad_id,next_day,label,account_id,altered,bid,create_ts,crow,exposure_time,industry_id,op_date,op_day,product_class,product_id,sizes,online_days
0,10003,20190222,2,6539,0,31,1547593423,area:7572,"281474976694272,281474976694272,28147497669427...",186,2019-02-21,20190115,13,9446,30,36
1,100046,20190309,2,22818,0,150,1546206412,area:3269,"281474976694272,281474976694272,28147497669427...",17,2019-03-08,20181230,13,20433,34,67
2,100046,20190310,1,22818,0,150,1546206412,area:3269,"281474976694272,281474976694272,28147497669427...",17,2019-03-09,20181230,13,20433,34,68
3,100046,20190314,1,22818,0,150,1546206412,area:3269,"281474976694272,281474976694272,28147497669427...",17,2019-03-13,20181230,13,20433,34,72
4,100091,20190219,5,1385,0,31,1546292099,area:7572,"281474976694272,281474976694272,28147497669427...",84,2019-02-18,20181231,13,24175,30,48


In [6]:
min_od, max_od = dataset.online_days.min(), dataset.online_days.max()
min_od, max_od

(-1, 1420)

In [7]:
import math
def norm_days(value, min_od=min_od, max_od=max_od):
    l_v = math.log10(value + 2)
    l_min = math.log10(min_od + 2)
    l_max = math.log10(max_od + 2)
    return (l_v - l_min) / (l_max - l_min)

dataset['od_ln'] = dataset.online_days.apply(norm_days, args=(min_od, max_od))
dataset.head()

,ad_id,next_day,label,account_id,altered,bid,create_ts,crow,exposure_time,industry_id,op_date,op_day,product_class,product_id,sizes,online_days,od_ln
0,10003,20190222,2,6539,0,31,1547593423,area:7572,"281474976694272,281474976694272,28147497669427...",186,2019-02-21,20190115,13,9446,30,36,0.501057
1,100046,20190309,2,22818,0,150,1546206412,area:3269,"281474976694272,281474976694272,28147497669427...",17,2019-03-08,20181230,13,20433,34,67,0.583225
2,100046,20190310,1,22818,0,150,1546206412,area:3269,"281474976694272,281474976694272,28147497669427...",17,2019-03-09,20181230,13,20433,34,68,0.585207
3,100046,20190314,1,22818,0,150,1546206412,area:3269,"281474976694272,281474976694272,28147497669427...",17,2019-03-13,20181230,13,20433,34,72,0.592861
4,100091,20190219,5,1385,0,31,1546292099,area:7572,"281474976694272,281474976694272,28147497669427...",84,2019-02-18,20181231,13,24175,30,48,0.538860


In [8]:
# test = pd.read_csv(DATA_DIR + 'test_sample.dat', sep='\t', header=None, names=HEADER_TEST_SAMPLE).replace(-1, 0)
test = pd.read_csv(DATA_DIR + 'Btest_sample_new.dat', sep='\t', header=None, names=HEADER_TEST_SAMPLE).replace(-1, 0)
test['label'] = 0
test['online_days'] = (pd.to_datetime('2019-03-21')  - pd.to_datetime(test.create_ts, unit='s')).dt.days
test.head()

,id,ad_id,create_ts,sizes,industry_id,product_class,product_id,account_id,exposure_time,crow,bid,label,online_days
0,1,152978,1480387159,30,118,3,0,23511,"281474976710655,281474976710655,28147497671065...","age:182,202,739,340,536,287,690,988,187,367,47...",37,0,841
1,2,507977,1551793928,30,122,13,629,4211,"70368744161280,70368744161280,70368744161280,7...",all,54,0,15
2,3,639943,1529565757,34,84,13,2525,15582,"281474976694272,281474976694272,28147497669427...","age:433,202,739,340,988,479,942,731,844,837,33...",161,0,272
3,4,605859,1553161934,40,98,13,3808,10907,"1099243192320,1099243192320,1099243192320,1099...",all,28,0,-1
4,5,742582,1551329520,40,122,13,7017,15855,"274936627200,274936627200,274936627200,2749366...","age:433,819,79,522,601,753,347,829,741",12,0,20


In [9]:
test['online_days'] = test.online_days.where(test['online_days'] >= -1,
                                            -1)
test.head()

,id,ad_id,create_ts,sizes,industry_id,product_class,product_id,account_id,exposure_time,crow,bid,label,online_days
0,1,152978,1480387159,30,118,3,0,23511,"281474976710655,281474976710655,28147497671065...","age:182,202,739,340,536,287,690,988,187,367,47...",37,0,841
1,2,507977,1551793928,30,122,13,629,4211,"70368744161280,70368744161280,70368744161280,7...",all,54,0,15
2,3,639943,1529565757,34,84,13,2525,15582,"281474976694272,281474976694272,28147497669427...","age:433,202,739,340,988,479,942,731,844,837,33...",161,0,272
3,4,605859,1553161934,40,98,13,3808,10907,"1099243192320,1099243192320,1099243192320,1099...",all,28,0,-1
4,5,742582,1551329520,40,122,13,7017,15855,"274936627200,274936627200,274936627200,2749366...","age:433,819,79,522,601,753,347,829,741",12,0,20


In [10]:
test['od_ln'] = test.online_days.apply(norm_days, args=(min_od, max_od))
test.head()

,id,ad_id,create_ts,sizes,industry_id,product_class,product_id,account_id,exposure_time,crow,bid,label,online_days,od_ln
0,1,152978,1480387159,30,118,3,0,23511,"281474976710655,281474976710655,28147497671065...","age:182,202,739,340,536,287,690,988,187,367,47...",37,0,841,0.927980
1,2,507977,1551793928,30,122,13,629,4211,"70368744161280,70368744161280,70368744161280,7...",all,54,0,15,0.390259
2,3,639943,1529565757,34,84,13,2525,15582,"281474976694272,281474976694272,28147497669427...","age:433,202,739,340,988,479,942,731,844,837,33...",161,0,272,0.773177
3,4,605859,1553161934,40,98,13,3808,10907,"1099243192320,1099243192320,1099243192320,1099...",all,28,0,-1,0.000000
4,5,742582,1551329520,40,122,13,7017,15855,"274936627200,274936627200,274936627200,2749366...","age:433,819,79,522,601,753,347,829,741",12,0,20,0.425774


In [11]:
total_exposure_math_fea = pd.read_hdf(DATA_DIR + 'total_exposure_math_fea.h5', 'total_exposure_math_fea')
total_exposure_math_fea.head()

ad_loc_id                                                            \
          count        mean        std    min    25%    50%    75%    max   
ad_id                                                                       
1          27.0  188.259259  52.691842   18.0  198.0  198.0  198.0  313.0   
2          75.0  146.560000  92.779343   39.0   56.0  123.0  196.0  297.0   
3          12.0  240.833333  47.668998  209.0  209.0  209.0  297.0  327.0   
4         602.0  208.162791  39.879854   30.0  199.0  209.0  209.0  323.0   
5           1.0   94.000000        NaN   94.0   94.0   94.0   94.0   94.0   

         bid                ...      total_ecpm               uid  \
       count        mean    ...             75%        max  count   
ad_id                       ...                                     
1       27.0  306.629630    ...      1027.25450   5311.008   27.0   
2       75.0  461.933333    ...      3365.49000  80089.520   75.0   
3       12.0   23.166667    ...       425.49675   7855.584   12.0   
4      602.0  173.282392    ...      1684.31250  13872.000  602.0   
5        1.0   12.000000    ...      8572.76000   8572.760    1.0   

                                                                    \
                mean            std       min        25%       50%   
ad_id                                                                
1      615381.777778  476662.410536    4784.0  119111.50  637025.0   
2      700073.173333  422250.156044     253.0  391392.00  695204.0   
3      644729.416667  426314.485286    6518.0  394688.75  666705.0   
4      686511.850498  398081.904489    3110.0  352644.75  685953.5   
5      260267.000000            NaN  260267.0  260267.00  260267.0   

                              
              75%        max  
ad_id                         
1       965421.50  1390924.0  
2       979732.00  1371564.0  
3       833585.75  1363790.0  
4      1022921.25  1395382.0  
5       260267.00   260267.0  

[5 rows x 72 columns]

In [12]:
total_exposure_math_fea.columns

MultiIndex(levels=[[u'ad_loc_id', u'bid', u'expo_size', u'pctr', u'quality_ecpm', u'req_id', u'req_ts', u'total_ecpm', u'uid'], [u'count', u'mean', u'std', u'min', u'25%', u'50%', u'75%', u'max']],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8], [0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7]])

In [13]:
# 加入统计特征
def col_with_prefix(col, prefix, ignore_cols=['ad_id']):
    if col in ignore_cols:
        return col
    return prefix + col
ecpm_fea = total_exposure_math_fea['quality_ecpm'].reset_index()
ecpm_fea.columns = [col_with_prefix(col, 'quality_ecpm_') for col in ecpm_fea.columns]
tmp = total_exposure_math_fea['total_ecpm'].reset_index()
tmp.columns = [col_with_prefix(col, 'total_ecpm_') for col in tmp.columns]
ecpm_fea = ecpm_fea.merge(tmp, on='ad_id', how='left')
ecpm_fea.head()

,ad_id,quality_ecpm_count,quality_ecpm_mean,quality_ecpm_std,quality_ecpm_min,quality_ecpm_25%,quality_ecpm_50%,quality_ecpm_75%,quality_ecpm_max,total_ecpm_count,total_ecpm_mean,total_ecpm_std,total_ecpm_min,total_ecpm_25%,total_ecpm_50%,total_ecpm_75%,total_ecpm_max
0,1,27.0,59.071111,27.363017,0.00,41.360,56.94,71.260,139.20,27.0,994.557815,1401.232530,63.700,202.0525,418.304,1027.25450,5311.008
1,2,75.0,133.569333,164.785034,0.00,55.330,114.34,172.490,1396.52,75.0,3625.049333,9452.879593,331.440,938.7200,1407.600,3365.49000,80089.520
2,3,12.0,320.943333,501.601407,54.50,116.425,133.27,155.625,1785.36,12.0,997.404917,2197.574602,95.375,202.4555,216.482,425.49675,7855.584
3,4,602.0,150.149601,115.652356,0.00,84.645,117.26,181.580,1241.42,602.0,1423.875194,1487.652861,2.120,619.4615,963.364,1684.31250,13872.000
4,5,1.0,5361.760000,NaN,5361.76,5361.760,5361.76,5361.760,5361.76,1.0,8572.760000,NaN,8572.760,8572.7600,8572.760,8572.76000,8572.760


In [14]:
to_fillna_v = ecpm_fea.mean()
# std有None, 为了避免在test中被filla成其他的，应该先自身fillna, 但应该在fillna统计量之前
ecpm_fea.fillna(0.0, inplace=True)
train = dataset.merge(ecpm_fea, on=['ad_id'], suffixes=('', '_y'), how='left')
dub_cols = train.filter(regex='_y$').columns
train.drop(columns=dub_cols, axis=1, inplace=True)
train.head()

,ad_id,next_day,label,account_id,altered,bid,create_ts,crow,exposure_time,industry_id,...,quality_ecpm_75%,quality_ecpm_max,total_ecpm_count,total_ecpm_mean,total_ecpm_std,total_ecpm_min,total_ecpm_25%,total_ecpm_50%,total_ecpm_75%,total_ecpm_max
0,10003,20190222,2,6539,0,31,1547593423,area:7572,"281474976694272,281474976694272,28147497669427...",186,...,86.78,146.14,18.0,381.241111,285.406845,38.88,156.52,327.53,598.50,1022.98
1,100046,20190309,2,22818,0,150,1546206412,area:3269,"281474976694272,281474976694272,28147497669427...",17,...,175.48,722.76,5.0,1973.054000,2354.543955,649.40,758.20,822.63,1491.58,6143.46
2,100046,20190310,1,22818,0,150,1546206412,area:3269,"281474976694272,281474976694272,28147497669427...",17,...,175.48,722.76,5.0,1973.054000,2354.543955,649.40,758.20,822.63,1491.58,6143.46
3,100046,20190314,1,22818,0,150,1546206412,area:3269,"281474976694272,281474976694272,28147497669427...",17,...,175.48,722.76,5.0,1973.054000,2354.543955,649.40,758.20,822.63,1491.58,6143.46
4,100091,20190219,5,1385,0,31,1546292099,area:7572,"281474976694272,281474976694272,28147497669427...",84,...,50.52,53.32,5.0,271.684000,84.192955,202.44,214.34,214.76,353.64,373.24


In [15]:
# to_fillna_dict = dict(zip(ad_fea.select('number').columns, ))
to_fillna_dict = to_fillna_v.to_dict()
test = test.merge(ecpm_fea, on=['ad_id'], suffixes=('', '_y'), how='left').fillna(to_fillna_dict)
test.drop(columns=dub_cols, axis=1, inplace=True)
test.head()

,id,ad_id,create_ts,sizes,industry_id,product_class,product_id,account_id,exposure_time,crow,...,quality_ecpm_75%,quality_ecpm_max,total_ecpm_count,total_ecpm_mean,total_ecpm_std,total_ecpm_min,total_ecpm_25%,total_ecpm_50%,total_ecpm_75%,total_ecpm_max
0,1,152978,1480387159,30,118,3,0,23511,"281474976710655,281474976710655,28147497671065...","age:182,202,739,340,536,287,690,988,187,367,47...",...,24.560000,110.16000,213.000000,46.773357,45.307551,5.928000,18.800000,29.532000,55.260000,253.368000
1,2,507977,1551793928,30,122,13,629,4211,"70368744161280,70368744161280,70368744161280,7...",all,...,60.950000,1545.66000,423.000000,183.979660,358.179219,7.038000,61.735500,96.441000,166.693500,3941.433000
2,3,639943,1529565757,34,84,13,2525,15582,"281474976694272,281474976694272,28147497669427...","age:433,202,739,340,988,479,942,731,844,837,33...",...,551.330000,1432.52000,82.000000,2907.256220,2329.434992,192.983000,867.993000,2719.265000,3915.003750,10027.640000
3,4,605859,1553161934,40,98,13,3808,10907,"1099243192320,1099243192320,1099243192320,1099...",all,...,473.235985,1865.74607,199.316243,2222.409797,2580.197341,683.215436,1026.372725,1439.884016,2376.079501,13398.234322
4,5,742582,1551329520,40,122,13,7017,15855,"274936627200,274936627200,274936627200,2749366...","age:433,819,79,522,601,753,347,829,741",...,473.235985,1865.74607,199.316243,2222.409797,2580.197341,683.215436,1026.372725,1439.884016,2376.079501,13398.234322


In [16]:
test.columns

Index([u'id', u'ad_id', u'create_ts', u'sizes', u'industry_id',
       u'product_class', u'product_id', u'account_id', u'exposure_time',
       u'crow', u'bid', u'label', u'online_days', u'od_ln',
       u'quality_ecpm_count', u'quality_ecpm_mean', u'quality_ecpm_std',
       u'quality_ecpm_min', u'quality_ecpm_25%', u'quality_ecpm_50%',
       u'quality_ecpm_75%', u'quality_ecpm_max', u'total_ecpm_count',
       u'total_ecpm_mean', u'total_ecpm_std', u'total_ecpm_min',
       u'total_ecpm_25%', u'total_ecpm_50%', u'total_ecpm_75%',
       u'total_ecpm_max'],
      dtype='object')

## 加入覆盖人群总量特征
> 先由single特征计算匹配crow数，再由multi特征计算覆盖率，最后两者相乘得到预估覆盖人数

In [30]:
def parse_crow_match_cnt(line):
    crow, cnt = line.split('\t')
    return crow, int(cnt)
crow_single_match_dict = dict([parse_crow_match_cnt(line)
                              for line in open(DATA_DIR + 'dataset_crow_single_match.out')])
print(len(crow_single_match_dict))

4073


In [31]:
# crow_single_match_num
train['csmn'] = train.crow.map(crow_single_match_dict)
train[['crow', 'csmn']].head()

,crow,csmn
0,area:7572,1396718
1,area:3269,1396718
2,area:3269,1396718
3,area:3269,1396718
4,area:7572,1396718


In [32]:
test['csmn'] = test.crow.map(crow_single_match_dict)

In [33]:
from collections import defaultdict
# key遵循json规范，都为字符串
crow_multi_match_dict = defaultdict(dict)
for line in open(DATA_DIR + 'multi_user_col_cnt.tsv'):
    col, value, rate = line.strip('\n').split('\t')
    crow_multi_match_dict[col][value] = float(rate)

In [34]:
crow_multi_match_dict.get('area', {}).get('1231')

0.993975161772

In [35]:
def cal_multi_match_prob(crow, crow_multi_match_dict):
    res = 1.0
    if crow == 'all':
        return res
    kvs = crow.split('|')
    for kv in kvs:
        k, values = kv.split(':')
        if k not in crow_multi_match_dict:
            continue
        sub_rate = None
        sub_dict = crow_multi_match_dict[k]
        for v in values.split(','):
            _r = sub_dict.get(v, 1)
            if sub_rate is None:
                sub_rate = _r
            else:
                sub_rate = sub_rate + _r - sub_rate * _r  # 同一类别取并集
        res *= sub_rate  # 不同类别取交集
    return res
print(cal_multi_match_prob('area:1231|behavior:19137,1170,39938,14452,22585,13412,21552,16749,468,1126,26904,2152,750,8673,1296,16258,13945,26908,22581,33712,8992,36111,27671,23654,27035,27306,26848', crow_multi_match_dict))
print(cal_multi_match_prob('age:601,79,202,942,394,347,847,731,393,829,787,229,333,741,608,479,433,1,753,522,819', crow_multi_match_dict))

0.693111231804
1.0


In [36]:
# crow_multi_match_rate
train['cmmr'] = train.crow.apply(lambda x: cal_multi_match_prob(x, crow_multi_match_dict))
test['cmmr'] = test.crow.apply(lambda x: cal_multi_match_prob(x, crow_multi_match_dict))
train.head()

,ad_id,next_day,label,account_id,altered,bid,create_ts,crow,exposure_time,industry_id,...,total_ecpm_mean,total_ecpm_std,total_ecpm_min,total_ecpm_25%,total_ecpm_50%,total_ecpm_75%,total_ecpm_max,csmn,cmmr,pcmn
0,10003,20190222,2,6539,0,31,1547593423,area:7572,"281474976694272,281474976694272,28147497669427...",186,...,381.241111,285.406845,38.88,156.52,327.53,598.50,1022.98,1396718,0.021994,30719.0
1,100046,20190309,2,22818,0,150,1546206412,area:3269,"281474976694272,281474976694272,28147497669427...",17,...,1973.054000,2354.543955,649.40,758.20,822.63,1491.58,6143.46,1396718,0.018453,25774.0
2,100046,20190310,1,22818,0,150,1546206412,area:3269,"281474976694272,281474976694272,28147497669427...",17,...,1973.054000,2354.543955,649.40,758.20,822.63,1491.58,6143.46,1396718,0.018453,25774.0
3,100046,20190314,1,22818,0,150,1546206412,area:3269,"281474976694272,281474976694272,28147497669427...",17,...,1973.054000,2354.543955,649.40,758.20,822.63,1491.58,6143.46,1396718,0.018453,25774.0
4,100091,20190219,5,1385,0,31,1546292099,area:7572,"281474976694272,281474976694272,28147497669427...",84,...,271.684000,84.192955,202.44,214.34,214.76,353.64,373.24,1396718,0.021994,30719.0


In [37]:
# predicted_crow_match_num, 之后可增加时间段match比例特征
train['pcmn'] = train['cmmr'] * train['csmn']
test['pcmn'] = test['cmmr'] * test['csmn']

In [38]:
# 注意不识别的为0，所以最大值还要加一作为长度
# max:
# ('status', 18)
# ('work', 6)
# ('behavior', 42459)  # 32631
# ('area', 14613) # 13646
# ('gender', 3)
# ('age', 988)  # user表实际就92个值？且div 12 mod 12有实际意义
# ('connectionType', 5)
# ('consuptionAbility', 3)
# ('education', 8)
# ('os', 4)

user_fea_max = [('gender', 3), ('age', 988), ('connectionType', 5), ('consuptionAbility', 3), ('education', 8), ('os', 4),
                   ('status', 18), ('work', 6), ('behavior', 42459), ('area', 14613)]
fea_id_max = dict(user_fea_max)
ind = 1
idx_start_dict = {'bid': 0}
for col, mx in user_fea_max:
    idx_start_dict[col] = ind
    ind += mx + 1
# 暂时将sizes列视为一列整数
single_id_cols = ['sizes', 'industry_id', 'product_class', 'product_id', 'account_id']
for col in single_id_cols:
    idx_start_dict[col] = ind
    mx_value = dataset[col].max()
    ind += mx_value + 1
    fea_id_max[col] = mx_value
idx_start_dict['exposure_time'] = ind
fea_id_max['exposure_time'] = 7 * 48 - 1

print(ind)

ind += 7 * 48
idx_start_dict['od_ln'] = ind

ad_fea_cols = ['quality_ecpm_mean', 'quality_ecpm_std', 'quality_ecpm_25%', 'quality_ecpm_50%', 'quality_ecpm_75%',
               'total_ecpm_mean', 'total_ecpm_std', 'total_ecpm_25%', 'total_ecpm_50%', 'total_ecpm_75%',
               'cmmr', 'pcmn'
              ]

print ind
for ad_fea_col in ad_fea_cols:
    ind += 1
    idx_start_dict[ad_fea_col] = ind


id_set = {'sizes', 'industry_id', 'product_class', 'product_id', 'account_id'}
# todo: 超出界限的不能加入特征，因为用户表中没有对应的id
# 需要排序后再输出
def make_libsvm(values, col, idx, id_set, id_len):
    if col in id_set:
        # 还要去重
        sorted_int_values = sorted(set((int(v) + idx for v in values.strip(',').split(',')
                                       if int(v) <= id_len
                                       )))
        return ' '.join(('%s:1' % v for v in sorted_int_values))
    if col in ['bid', 'od_ln'] + ad_fea_cols:
        return '%s:%s' % (idx, values)
    if col == 'exposure_time':
        bin_values = ''.join([bin(int(x))[2:] for x in values.strip(',').split(',')])
        return ' '.join(['%s:%s' % (idx + int(k), v)
                         for k, v in enumerate(bin_values) if v != '0'])
    if col == 'crow':
        pass
    raise Exception('unknow col: %s' % col)

user_fea_col = [col for col, _ in user_fea_max]
user_fea_id_set = set(user_fea_col)

sep_label_fea=' '
all_crow_v = '|'.join(['%s:0' % col for col in user_fea_col])
# 要按照从小到大的顺序来, crow里面也应如此; 注意single_id_cols是否包含sizes
# 暂时去掉 crow
fea_cols = ['bid', ] + single_id_cols + ['exposure_time', 'od_ln'] + ad_fea_cols

def _get_svm_line(tup):
    line = str(tup.label)
    # 人群特征需要单独处理
    for idx, col in enumerate(fea_cols):
        if col == 'crow':
            crow = tup.crow
            if crow == 'all':
                crow = all_crow_v
            kvs = {}
            for crow_kv in crow.split('|'):
                crow_col, values = crow_kv.split(':')
                kvs[crow_col] = values
            # 未定义的user_fea为0
            for crow_col in user_fea_col:
                idx_plus = idx_start_dict[crow_col]
                values = kvs.get(crow_col, '0')
                id_len = fea_id_max[crow_col]
                line += ' ' + make_libsvm(values, crow_col, idx_plus, user_fea_id_set, id_len)
        else:
            values = str(tup[idx])
            idx_plus = idx_start_dict[col]
            line += ' ' + make_libsvm(values, col, idx_plus, id_set, id_len=fea_id_max.get(col, 0))  # 'bid'等无max
            
    return line

121123
121459


In [39]:
with open(DATA_DIR + 'data.libsvm', 'w') as f:
    for tup in train[fea_cols + ['label']].itertuples(index=False):
        f.write('%s\n' % _get_svm_line(tup))

In [40]:
with open(DATA_DIR + 'test.libsvm', 'w') as f:
    for tup in test[fea_cols + ['label']].itertuples(index=False):
        f.write('%s\n' % _get_svm_line(tup))

In [41]:
idx_start_dict

{'account_id': 91380,
 'age': 5,
 'area': 43504,
 'behavior': 1044,
 'bid': 0,
 'cmmr': 121470,
 'connectionType': 994,
 'consuptionAbility': 1000,
 'education': 1004,
 'exposure_time': 121123,
 'gender': 1,
 'industry_id': 58185,
 'od_ln': 121459,
 'os': 1013,
 'pcmn': 121471,
 'product_class': 58437,
 'product_id': 58456,
 'quality_ecpm_25%': 121462,
 'quality_ecpm_50%': 121463,
 'quality_ecpm_75%': 121464,
 'quality_ecpm_mean': 121460,
 'quality_ecpm_std': 121461,
 'sizes': 58118,
 'status': 1018,
 'total_ecpm_25%': 121467,
 'total_ecpm_50%': 121468,
 'total_ecpm_75%': 121469,
 'total_ecpm_mean': 121465,
 'total_ecpm_std': 121466,
 'work': 1037}

In [42]:
for col in user_fea_col:
    print(col, idx_start_dict[col])

('gender', 1)
('age', 5)
('connectionType', 994)
('consuptionAbility', 1000)
('education', 1004)
('os', 1013)
('status', 1018)
('work', 1037)
('behavior', 1044)
('area', 43504)
